# Set up model and hyperparameters

In [3]:
from models.WaveNetVAE.WaveVae import WaveNetVAE
from models.WaveNetVAE.WVData import WVDataset
import torch
from torch.utils.data import DataLoader

"""
Hyperparameters
"""

learning_rate = 0.000001
batchsize = 4
device='cuda'
input_size = (40, 112)
upsamples = [2, 2, 2, 2, 2, 2, 2, 2]
zsize = 32

WaveVAE = WaveNetVAE(input_size,
                     num_hiddens = 768,
                     upsamples = upsamples,
                     zsize = zsize)

WaveVAE.to(device)

VAEDataset = WVDataset(audio_path = "../ConvDenoiser/testdatawav",
                       length = 4096,
                       sample_rate = 24000,
                       hop_length = 128)

VAEDataloader = DataLoader(VAEDataset,
                           batch_size = batchsize,
                           shuffle = True)

optimizer = torch.optim.AdamW(WaveVAE.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

WaveNet Receptive Field:  4093
4096


C:\Users\woute\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torchaudio\functional\functional.py:539: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(
Loading and preprocessing files to dataset.: 100%|██████████| 33/33 [03:00<00:00,  5.48s/it]


# Test tensor

In [5]:
onehot, mfcc, target = next(iter(VAEDataloader))
onehot = onehot.to(device)
mfcc = mfcc.to(device)
target = target.to(device)

print("Onehot size: ", onehot.size(), "| MFCC size: ", mfcc.size(), "| Target size: ", target.dtype)

output = WaveVAE(onehot, mfcc, True)
print(output[:, :, -1].dtype)
loss = loss_fn(output[:, :, -1], target)



Onehot size:  torch.Size([4, 256, 4096]) | MFCC size:  torch.Size([4, 40, 33]) | Target size:  torch.float64


RuntimeError: Input type (torch.cuda.DoubleTensor) and weight type (torch.cuda.FloatTensor) should be the same